In [1]:
import numpy as np
indices = np.random.choice(10, 5, replace=True)
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(arr[indices])
print(arr[-11:])
reward_buffer = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1]
size = 7
game_start_index = 0
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
game_start_index = 7
size = 15
print(len(reward_buffer))
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
print(len(reward_buffer))
overflow_reward = 1
overflow_count = 1

[5 5 3 2 7]
[ 1  2  3  4  5  6  7  8  9 10]
[ 1 -1  1 -1  1 -1  1]
[1, -1, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1]
15
[ 1 -1  1 -1  1 -1  1 -1]
[1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1]
15


In [2]:
# reward_buffer = np.array([1, -1, 1, -1, 1, -1, 0])
# reward_buffer[6] = [1, 1, 1, 1]
# print(reward_buffer)

In [3]:
# from alphazero_agent import AlphaZeroAgent
import gymnasium as gym 
import tensorflow as tf
import numpy as np
# import gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
env = TicTacToeEnv()
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
env = gym.wrappers.FrameStack(env, 4)

config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer_function': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.2,
        'adam_epsilon': 1e-7,
        'clipnorm': 0.5,
        # NORMALIZATION?
        # REWARD CLIPPING
        'num_epochs': 30,
        'num_filters': 32,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 5,
        'critic_conv_filters': 32,
        'critic_conv_layers': 1,
        'critic_dense_size': 32,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32,
        'actor_conv_layers': 1,
        'actor_dense_size': 32,
        'actor_dense_layers': 1,
        'memory_size': 180,
        'max_game_length': 9,
        'batch_size': 9,
        'dirichlet_alpha': 0.3,
        'dirichlet_epsilon': 0.25,
        'c_puct': 1,
        'monte_carlo_simulations': 18,
        'two_player': True,
        'weight_decay': 1e-4,
    }    



/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'TicTacToeEnv' is not defined

In [ ]:
agent = AlphaZeroAgent(env, config=config)
agent.train()

In [ ]:
array = np.zeros((3, 9))
array[0] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(array)

In [ ]:
import gym_envs
import gymnasium as gym
env = gym.make('gym_envs/TicTacToe-v0')

state, info = env.reset()
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
env.render()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(8)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(5)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
env.render()


env.reset()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
print("Reward:", reward)



In [ ]:
# import tensorflow as tf
# regularizer = tf.keras.regularizers.L2(2)
# print(regularizer(agent.model.trainable_variables))

In [ ]:
import numpy as np
import tensorflow as tf
arr = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

print(arr/np.sum(arr))
# print(tf.nn.softmax(arr))

In [1]:
from alphazero_agent import AlphaZeroAgent
# from alphazero_agent import AlphaZeroAgent
import gymnasium as gym
import tensorflow as tf
import numpy as np
import gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
# env = TicTacToeEnv(render_mode="rgb_array")
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
# env = gym.wrappers.FrameStack(env, 4)
env = gym.make('gym_envs/TicTacToe-v0', render_mode="rgb_array")


# MODEL SEEMS TO BE UNDERFITTING SO TRY AND GET IT TO OVERFIT THEN FIND A HAPPY MEDIUM
# 1. INCREASE THE NUMBER OF RESIDUAL BLOCKS
# 2. INCREASE THE NUMBER OF FILTERS
# 3. DECREASE REGULARIZATION
# 4. TRY DECREASING LEARNING RATE (maybe its that whole thing where the policy goes to like 1 0 0 0 0... etc and then goes back on the third training step, so maybe the learning rate is too high)
# 5. TO OVERFIT USE LESS DATA (but that is probably just a bad idea)
# config = {
#         'activation': 'relu',
#         'kernel_initializer': 'glorot_uniform',
#         'optimizer': tf.keras.optimizers.legacy.Adam,
#         'learning_rate': 0.001, # 0.00001 could maybe increase by a factor of 10 or 100 and try to do some weights regularization
#         'adam_epsilon': 3.25e-6,
#         'clipnorm': None,
#         # NORMALIZATION?
#         # REWARD CLIPPING
#         'training_steps': 40,
#         'num_filters': 256,
#         'kernel_size': 3,
#         'stride': 1,
#         'num_res_blocks': 20,
#         'critic_conv_filters': 32, # 1
#         'critic_conv_layers': 1,
#         'critic_dense_size': 256,
#         'critic_dense_layers': 1,
#         'actor_conv_filters': 32, # 
#         'actor_conv_layers': 1,
#         'actor_dense_size': 0,
#         'actor_dense_layers': 0,
#         'replay_buffer_size': 800, # IN GAMES
#         'replay_batch_size': 50, # IN MOVES
#         'root_dirichlet_alpha': 0.5, # 2 in theory?
#         'root_exploration_fraction': 0, # 0.25 in paper
#         'pb_c_base': 500,
#         'pb_c_init': 2,
#         'num_simulations': 200,
#         # 'two_player': True,
#         'weight_decay': 0.00, # could try setting this to something other than 0 and increasing learning rate
#         'num_sampling_moves': 0, 
#         'initial_temperature': 1,
#         'exploitation_temperature': 0.1,
#         'value_loss_factor': 1, # could try setting this to something other than 1
#         'games_per_generation': 10, # times 8 from augmentation
#     }

config = {
        'activation': 'relu',
        'kernel_initializer': 'glorot_uniform',
        'optimizer': tf.keras.optimizers.legacy.Adam,
        'min_learning_rate': 0.0005, #0.0001 # 0.00001 could maybe increase by a factor of 10 or 100 and try to do some weights regularization
        'max_learning_rate': 0.0005, #0.0001
        'number_of_lr_cycles': 1, # this will determine the step size based on training steps
        # STILL ADD A SCHEDULE FOR BASE LEARNING RATE (MIN LEARNING RATE)
        'adam_epsilon': 3.25e-6,
        'clipnorm': None,
        # NORMALIZATION?
        # REWARD CLIPPING
        'training_steps': 100, # alpha zero did 700,000, the lessons from alpha zero did 40 generations but 1000 batches per generation, so 40,000 batches (they just had a cyclical learning rate per generation (also they trained twice on the same data every generation))
        'num_filters': 256,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 20,
        'critic_conv_filters': 32, # 1
        'critic_conv_layers': 1,
        'critic_dense_size': 256,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32, # 
        'actor_conv_layers': 1,
        'actor_dense_size': 0,
        'actor_dense_layers': 0,
        'replay_buffer_size': 800, # IN GAMES
        'replay_batch_size': 560, # SHOULD BE ROUGHLY SAME AS AVERAGE MOVE PER GENERATION (SO LIKE 7 TIMES NUMBER OF GAMES PLAYED PER GENERATION) <- what was used in the original paper (they played 44M games, 50 moves per game and sampled 700,000 minibatches of size 4096 (so thats like sampling 1 time per move roughly but this was also happening with parrallel data collection i believe))
        'games_per_generation': 10, # times 8 from augmentation
        'root_dirichlet_alpha': 2.0, # Less than 1 more random, greater than one more flat # 2 in theory? # 0.3 in alphazero for chess # TRY CHANGING (MAYBE LOWER? (IT SEEMS TO PLAY THE SAME LINE OVER AND OVER AGAIN <- so we want a lesss flat distribution maybe)
        'root_exploration_fraction': 0.25, # 0.25 in paper
        'pb_c_base': 20000, # Seems unimportant to be honest (increases puct the more simulations there are)
        'pb_c_init': 1.25, # 1.25 in paper # MAYBE HIGHER? (IT SEEMS TO PLAY THE SAME LINE OVER AND OVER AGAIN)
        'num_simulations': 800, # INCREASE THIS since the model is missing 1 move wins (and also 2 and 3 move wins (it wins by luck)))
        # 'two_player': True,
        'weight_decay': 0.00001, # could try setting this to something other than 0 and increasing learning rate
        'num_sampling_moves': 4, 
        'initial_temperature': 1,
        'exploitation_temperature': 0.1,
        'value_loss_factor': 1, # could try setting this to something other than 1
    }

agent = AlphaZeroAgent(env, config=config, name="alphazero")
agent.train()

/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1 Physical GPUs, 1 Logical GPUs


2024-03-18 00:45:26.403055: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-18 00:45:26.403077: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-18 00:45:26.403083: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-18 00:45:26.403120: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-18 00:45:26.403140: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Training Step  1


/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

Training Step  2
Predicted Policy  [0.10769542 0.1104171  0.10975769 0.11311129 0.11293018 0.11287638
 0.10568353 0.11192612 0.11560226]
Predicted Value  0.029582686722278595
Target Policy [0.05625    0.05625    0.055      0.12375    0.06875    0.57125002
 0.05125    0.00875    0.00875   ]
Predicted Policy  [4.92727679e-07 5.98915387e-04 1.13545735e-04 5.55238798e-02
 0.00000000e+00 1.61536708e-02 3.74369975e-08 3.40242230e-04
 9.27269220e-01]
Predicted Value  0.9999228715896606
Target Policy [0.75       0.02       0.13124999 0.04125    0.         0.02
 0.00625    0.005      0.02625   ]
Predicted Policy  [1.1218580e-03 2.5457483e-02 0.0000000e+00 1.9333862e-01 0.0000000e+00
 1.0923699e-01 3.8161274e-04 1.9705629e-02 6.5075779e-01]
Predicted Value  0.9761841893196106
Target Policy [0.71625 0.04    0.      0.11    0.      0.0775  0.02625 0.00875 0.02125]
Predicted Policy  [0.0000000e+00 3.6781971e-04 0.0000000e+00 4.8554108e-02 0.0000000e+00
 1.2845799e-02 1.2620733e-08 1.9923771e-04 9.3

/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/alphazero/videos/alphazero folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/alphazero/videos/alphazero/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/alphazero/videos/alphazero/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/alphazero/videos/alphazero/rl-video-episode-0.mp4


Training Step  6
Predicted Policy  [0.11150451 0.11125997 0.11037945 0.11144686 0.11162298 0.11064539
 0.11069179 0.11177345 0.11067563]
Predicted Value  0.0037358980625867844
Target Policy [0.63875002 0.08       0.07875    0.05125    0.0525     0.04
 0.0325     0.01375    0.0125    ]
Predicted Policy  [0.12181813 0.13532658 0.         0.12847258 0.13319096 0.11421715
 0.1181306  0.135137   0.11370697]
Predicted Value  0.24026232957839966
Target Policy [0.85874999 0.0275     0.         0.02875    0.02125    0.025
 0.0125     0.0125     0.01375   ]
Predicted Policy  [0.         0.14372437 0.         0.1447406  0.14558217 0.14081831
 0.14136027 0.1452399  0.13853438]
Predicted Value  0.025068670511245728
Target Policy [0.      0.01625 0.      0.01625 0.02    0.90125 0.01875 0.01375 0.01375]
Predicted Policy  [0.         0.17801048 0.         0.16945907 0.17607716 0.
 0.15351938 0.17501159 0.1479223 ]
Predicted Value  0.2351093292236328
Target Policy [0.         0.03125    0.         0.89